## SHapley Additive exPlanations (SHAP)

### Library Imports 

In [ ]:
import os
import pickle

import pandas as pd
import shap

In [ ]:
RAND_STATE = 0

### Import the Train and Test Sets

In [ ]:
dataset_folder = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/datasets"

In [ ]:
X_train = pd.read_csv(os.path.join(dataset_folder, "obesity_X_train.csv"), index_col=0)

In [ ]:
X_test = pd.read_csv(os.path.join(dataset_folder, "obesity_X_test.csv"), index_col=0)

### Import Random Forest Model

In [ ]:
def import_model(filename):
    file_path = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/models/{filename}"
    with open(file_path, 'rb') as file: 
        model = pickle.load(file)
    print(f"Model imported from {file_path}")
    return model

In [ ]:
rand_forest = import_model("rand_forest.pkl")

In [ ]:
rand_forest_no_weight = import_model("rand_forest_no_weight.pkl")

### Preprocessing

In [ ]:
nominal_features = [
    "Gender",
    "family_history_with_overweight",
    "FAVC",
    "CAEC",
    "SMOKE",
    "SCC",
    "CALC",
    "MTRANS"
]

### Create SHAP explainer 

In [ ]:
explainer = shap.TreeExplainer(rand_forest)
shap_values = explainer.shap_values(X_test)

### Summary Plot

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.summary_plot(shap_values[0], X_test)

In [ ]:
shap_values[0]

In [ ]:
X_test